# Introdução ao statsmodels

Objetivos:
- Introduzir o Patsy (especificação de modelo similar ao R)
- Introduzir modelagem com o Statsmodels

## Índice <a name="topo"></a>

1. [Introdução ao Patsy](#1)
    1. [Definindo a matriz de modelagem no Patsy](#1.1)
    2. [Retirando o intercepto](#1.2)
    3. [Transformações de dados no Patsy](#1.3)
    4. [Aplicando patsy a dados novos](#1.4)
5. [Introdução ao statsmodels](#2)
    1. [Usando o API base do statsmodels](#2.1)
    2. [Usando o API *formula* do statsmodels](#2.2)

In [1]:
import pandas as pd
import numpy as np
from seaborn import load_dataset

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
tips = load_dataset("tips")
tips['tip_pct'] = tips['tip'] / (tips['total_bill'] - tips['tip'])
tips['net_bill'] = tips['total_bill'] - tips['tip']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct,net_bill
0,16.99,1.01,Female,No,Sun,Dinner,2,0.063204,15.98
1,10.34,1.66,Male,No,Sun,Dinner,3,0.191244,8.68
2,21.01,3.50,Male,No,Sun,Dinner,3,0.199886,17.51
3,23.68,3.31,Male,No,Sun,Dinner,2,0.162494,20.37
4,24.59,3.61,Female,No,Sun,Dinner,4,0.172069,20.98


## <span style="color:blue">1. Introdução ao Patsy</span><a name="1"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topo) </div>

O Patsy é um pacote que permite especificar uma matriz de modelagem de uma forma prática como no R, com a seguinte sintaxe:
```
'y ~ x1 + x2 + x3'
```

Ele possui diversos recursos como:
- inclui intercepto (ou exclui utilizando +0)
- inserir funções das variáveis de forma prática
- trabalhar com dados categorizados
- facilita a aplicação a novos dados

### <span style="color:blue">1.1 Definindo a matriz de modelagem com Patsy</span><a name="1.1"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topo) </div> 

In [3]:
y, X = patsy.dmatrices('tip_pct ~ sex + smoker + size + total_bill + 1', tips[:5])
X

DesignMatrix with shape (5, 5)
  Intercept  sex[T.Female]  smoker[T.No]  size  total_bill
          1              1             1     2       16.99
          1              0             1     3       10.34
          1              0             1     3       21.01
          1              0             1     2       23.68
          1              1             1     4       24.59
  Terms:
    'Intercept' (column 0)
    'sex' (column 1)
    'smoker' (column 2)
    'size' (column 3)
    'total_bill' (column 4)

In [4]:
y

DesignMatrix with shape (5, 1)
  tip_pct
  0.06320
  0.19124
  0.19989
  0.16249
  0.17207
  Terms:
    'tip_pct' (column 0)

### <span style="color:blue">1.2 Retirando o intercepto</span><a name="1.2"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topo) </div>  

Podemos retirar o intercepto no Patsy simplesmente com a instrução ```+0```. Ou, alternativamente, podemos colocá-lo de forma explícita com ```+1```.

In [5]:
y, X = patsy.dmatrices('tip_pct ~ sex + smoker + size + total_bill + 0', tips[:5])
X

DesignMatrix with shape (5, 5)
  sex[Male]  sex[Female]  smoker[T.No]  size  total_bill
          0            1             1     2       16.99
          1            0             1     3       10.34
          1            0             1     3       21.01
          1            0             1     2       23.68
          0            1             1     4       24.59
  Terms:
    'sex' (columns 0:2)
    'smoker' (column 2)
    'size' (column 3)
    'total_bill' (column 4)

### <span style="color:blue">1.3 Transformações de dados no Patsy</span><a name="1.3"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topo) </div>  

#### Aplicando funções

Podemos aplicar funções vetorizadas no Patsy de forma transparente, dentro do string. Em especial, a maioria das funções do Numpy são vetorizadas e podem ser usadas normalmente, como o ```np.log()``` e ```np.exp()``` que fazem logaritmo e função exponencial ($e^x$) respectivamente.

In [6]:
y, X = patsy.dmatrices('tip_pct ~ np.log(size) + standardize(total_bill) + center(tip)', tips[:5])
X


DesignMatrix with shape (5, 4)
  Intercept  np.log(size)  standardize(total_bill)  center(tip)
          1       0.69315                 -0.44762       -1.608
          1       1.09861                 -1.72407       -0.958
          1       1.09861                  0.32401        0.882
          1       0.69315                  0.83651        0.692
          1       1.38629                  1.01118        0.992
  Terms:
    'Intercept' (column 0)
    'np.log(size)' (column 1)
    'standardize(total_bill)' (column 2)
    'center(tip)' (column 3)

In [7]:

y, X = patsy.dmatrices('tip ~ sex + smoker + standardize(size) + center(total_bill)  + 0', tips[:5])
X

DesignMatrix with shape (5, 5)
  sex[Male]  sex[Female]  smoker[T.No]  standardize(size)  center(total_bill)
          0            1             1           -1.06904              -2.332
          1            0             1            0.26726              -8.982
          1            0             1            0.26726               1.688
          1            0             1           -1.06904               4.358
          0            1             1            1.60357               5.268
  Terms:
    'sex' (columns 0:2)
    'smoker' (column 2)
    'standardize(size)' (column 3)
    'center(total_bill)' (column 4)

#### Somando variáveis dentro do Patsy

Como o caractere ```+``` é interpretado dentro do contexto da construção da matriz de modelagem, se quisermos somar variáveis no Patsy podemos usar a função ```I()``` como abaixo. 



In [8]:
y, X = patsy.dmatrices('tip ~ I(size + total_bill) + size + total_bill', tips[:5])
X

DesignMatrix with shape (5, 4)
  Intercept  I(size + total_bill)  size  total_bill
          1                 18.99     2       16.99
          1                 13.34     3       10.34
          1                 24.01     3       21.01
          1                 25.68     2       23.68
          1                 28.59     4       24.59
  Terms:
    'Intercept' (column 0)
    'I(size + total_bill)' (column 1)
    'size' (column 2)
    'total_bill' (column 3)

#### Dados categorizados no Patsy

Já vimos que o Patsy calcula as variáveis dummy para variáveis tipo **str**. Mas se quisermos tratar uma variável numérica como categorizada temos que usar a função ```C()```.



In [9]:
y, X = patsy.dmatrices('tip ~ sex  + C(size) + total_bill', tips[:5])
X

DesignMatrix with shape (5, 5)
  Intercept  sex[T.Female]  C(size)[T.3]  C(size)[T.4]  total_bill
          1              1             0             0       16.99
          1              0             1             0       10.34
          1              0             1             0       21.01
          1              0             0             0       23.68
          1              1             0             1       24.59
  Terms:
    'Intercept' (column 0)
    'sex' (column 1)
    'C(size)' (columns 2:4)
    'total_bill' (column 4)

#### Definindo interações com Patsy
Podemos inserir interações no Patsy da mesma forma que em boa parte dos softwares estatísticos programáveis com a sintaxe ```var1*var2```.


In [10]:
y, X = patsy.dmatrices('tip ~ sex  + time + sex*time', tips[:5])
X

DesignMatrix with shape (5, 4)
  Intercept  sex[T.Female]  time[T.Dinner]  sex[T.Female]:time[T.Dinner]
          1              1               1                             1
          1              0               1                             0
          1              0               1                             0
          1              0               1                             0
          1              1               1                             1
  Terms:
    'Intercept' (column 0)
    'sex' (column 1)
    'time' (column 2)
    'sex:time' (column 3)

In [11]:
y, X = patsy.dmatrices('tip ~ sex:time + 0', tips[:5])
X

DesignMatrix with shape (5, 4)
  Columns:
    ['sex[Male]:time[Lunch]',
     'sex[Female]:time[Lunch]',
     'sex[Male]:time[Dinner]',
     'sex[Female]:time[Dinner]']
  Terms:
    'sex:time' (columns 0:4)
  (to view full data, use np.asarray(this_obj))

### <span style="color:blue">1.4 Aplicando patsy a dados novos</span><a name="1.4"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topo) </div>  

Como vimos, a função ```patsy.dmatrices()``` constroi a matriz de modelagem de acordo com as instruções do string que é passado como parâmetro. Com bastante frequência, vamos precisar aplicar o nosso modelo a dados novos, vamos portanto, precisar construir a mesma matriz de dados para esses dados novos. 

A função ```patsy.build_design_matrices()``` cumpre com essa tarefa de construir a matriz de modelagem para novos dados, de modo a facilitar bastante a implantação do modelo.

In [12]:
novos_Dados = tips[20:30]

new_X = patsy.build_design_matrices([X.design_info], novos_Dados)
new_X

[DesignMatrix with shape (10, 4)
   Columns:
     ['sex[Male]:time[Lunch]',
      'sex[Female]:time[Lunch]',
      'sex[Male]:time[Dinner]',
      'sex[Female]:time[Dinner]']
   Terms:
     'sex:time' (columns 0:4)
   (to view full data, use np.asarray(this_obj))]

## <span style="color:blue">2 Introdução ao Statsmodels</span><a name="2"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topo) </div>  


### <span style="color:blue">2.1 Usando a api do statsmodels</span><a name="2.1"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topo) </div>  

O statsmodels tem o API padrão, que é compatível com o Patsy, ou seja, você pode inserir a matriz de modelagem fornecida pelo ```patsy.dmatrices()```, bem como um objeto do numpy ou do pandas, como fizemos com o scikitlearn.

In [13]:
y, X = patsy.dmatrices('tip ~ sex  + time + smoker + net_bill', tips)
X

DesignMatrix with shape (244, 5)
  Intercept  sex[T.Female]  time[T.Dinner]  smoker[T.No]  net_bill
          1              1               1             1     15.98
          1              0               1             1      8.68
          1              0               1             1     17.51
          1              0               1             1     20.37
          1              1               1             1     20.98
          1              0               1             1     20.58
          1              0               1             1      6.77
          1              0               1             1     23.76
          1              0               1             1     13.08
          1              0               1             1     11.55
          1              0               1             1      8.56
          1              1               1             1     30.26
          1              0               1             1     13.85
          1              0   

In [14]:
modelo = sm.OLS(y,X).fit()

modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    tip   R-squared:                       0.335
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     30.13
Date:                Tue, 16 Jul 2024   Prob (F-statistic):           2.64e-20
Time:                        14:32:35   Log-Likelihood:                -375.13
No. Observations:                 244   AIC:                             760.3
Df Residuals:                     239   BIC:                             777.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.1999      0.242      4.952      0.000       0.723       1.677
sex[T.Female]     -0.0034      0.156     -0.022      0.982      -0.311       0.305
time[T.Dinner]     0.0584      0.168      0.347      0.729      -0.273       0.390
smoker[T.No]       0.1405      0.151      0.932      0.352      -0.156       0.437
net_bill           0.0995      0.009     10.662      0.000       0.081       0.118
==============================================================================
Omnibus:                       36.537   Durbin-Watson:                   2.121
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               65.875
Skew:                           0.800   Prob(JB):                     4.96e-15
Kurtosis:                       4.980   Cond. No.                         71.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### <span style="color:blue">2.2 Usando a api *formula* do statsmodels</span><a name="2.2"></a>

<div style="text-align: right"
     
[Voltar ao índice](#topo) </div>  

O statsmodels possui um pacote *formula*, que já integra a interface do Patsy. Com ele podemos indicar a sintaxe do modelo diretamente na função do modelo, ```ols()``` no caso. Repare que a função vem com letras minúsculas, em contraste com o API padrão do statsmodels, propositalmente, para deixar claro qual o API que está sendo utilizado.

In [15]:
results = smf.ols('''
                    tip_pct ~ sex 
                                + size 
                                + np.log(total_bill)
                  ''', tips).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                tip_pct   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     7.953
Date:                Tue, 16 Jul 2024   Prob (F-statistic):           4.46e-05
Time:                        14:32:35   Log-Likelihood:                 107.89
No. Observations:                 244   AIC:                            -207.8
Df Residuals:                     240   BIC:                            -193.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.5401      0.071      7.618      0.000       0.400       0.680
sex[T.Female]         -0.0105      0.021     -0.496      0.620      -0.052       0.031
size                   0.0176      0.013      1.345      0.180      -0.008       0.043
np.log(total_bill)    -0.1313      0.029     -4.589      0.000      -0.188      -0.075
==============================================================================
Omnibus:                      460.749   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           214315.295
Skew:                          10.735   Prob(JB):                         0.00
Kurtosis:                     146.594   Cond. No.                         31.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""